In [1]:
# !pip install rapidfuzz

In [2]:
from typing import Tuple, List, Dict, Any, Sequence
import polars as pl
import polars.selectors as cs
import pandas as pd
from pprint import pprint, pformat
import sys
from pathlib import Path
from tqdm import tqdm, trange
import psutil
import re


# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from src.utils import process_lazyframe_in_chunks
from src.loading import DataLoader
from src.preprocess import TextPreprocessor, create_udi_preprocessor, create_company_preprocessor, create_generic_preprocessor
from src.preprocess.preprocess import get_pattern_cols, \
    get_unique_by_cols_safe, get_unique, \
    analyze_null_values, replace_pattern_with_null, overview_col

In [3]:
# maude 데이터 불러오기
loader1 = DataLoader(
    start=2024,
    end=2024,
    output_file = DATA_DIR / 'output.parquet',
    max_workers=4
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
maude_lf = loader1.load(adapter=adapter, **polars_kwargs)
maude_lf


📖 /Users/t2023-m0030/Desktop/i_hate_maude/data/output.parquet 로딩 중... (adapter=polars)


In [4]:
IDENTIFIER_PATTERNS = [
    r"^device_\d+_brand_name$",
    r"identifiers_\d+_id", 
    r"identifiers_\d+_issuing_agency", 
    r"identifiers_\d+_package_discontinue_date", 
    r"identifiers_\d+_package_status", 
    r"identifiers_\d+_package_type", 
    r"identifiers_\d+_quantity_per_package", 
    r"identifiers_\d+_type", 
    r"identifiers_\d+_unit_of_use_id"
]
UDI_DI_PATTERNS = [r'^identifiers_\d+_id$']
TYPE_PATTERNS = [
    r'identifiers_\d+_type'
]

CUSTOMER_PATTERNS = [r'^customer']
DEVICE_SIZE_PATTERNS = [r'^device_sizes']
STORAGE_PATTERNS = [r'^storage']

# 열 drop

In [5]:
# 기본 변수
BASE_COLS = [
    'mdr_report_key', 'report_number', 'adverse_event_flag', 'product_problem_flag', 
    'date_of_event', 'date_received', 'device_date_of_manufacturer', 'event_type',
    'previous_use_code', 'single_use_flag', 'report_source_code',
    'reprocessed_and_reused_flag', 'report_to_fda', 'event_location', 
    'manufacturer_link_flag',
    'pma_pmn_number'
]

DEVICE_COLS = [
    "device_0_manufacturer_d_name",
    "device_0_manufacturer_d_postal_code",
    "device_0_brand_name",
    "device_0_catalog_number",
    "device_0_model_number",
    "device_0_udi_di",
    "device_0_lot_number",
    "device_0_udi_public",
    "device_0_device_report_product_code",
    "device_0_device_operator",
    "device_0_openfda_device_class",
    "device_0_openfda_device_name",
]



PATIENT_COLS = [
    "patient_0_patient_age",
    "patient_0_patient_sex",
    "patient_0_patient_weight",
    "patient_0_patient_race",
    "patient_0_patient_problems",
    "patient_0_sequence_number_outcome",
    "patient_0_sequence_number_treatment",
]



MDR_TEXT_PATTERNS = [
    r"^mdr_text_.*_text$",
    r"^mdr_text_.*_text_type_code$",
]

MDR_COLS = get_pattern_cols(maude_lf, MDR_TEXT_PATTERNS)
TOTAL_COLS = BASE_COLS + DEVICE_COLS + PATIENT_COLS + MDR_COLS

In [6]:
drop_patterns = CUSTOMER_PATTERNS + DEVICE_SIZE_PATTERNS + STORAGE_PATTERNS

regex = "|".join(drop_patterns)


In [7]:
maude_lf = maude_lf.select(TOTAL_COLS)

# device 3만 선택 (필터링)

In [8]:
maude_lf = maude_lf.filter(
    pl.col('device_0_openfda_device_class').eq('3')
)

# 클렌징

In [9]:
maude_step1_path = DATA_DIR / 'silver' / 'clean_step1.parquet'
maude_step2_path = DATA_DIR / 'silver' / 'clean_step2.parquet'
maude_step3_path = DATA_DIR / 'silver' / 'clean_step3.parquet'

### (+추가) 지영님 1:1 매칭  
- device_0_device_name과 product_code에서 최빈값으로 join

In [10]:
def device_name_clean(maude_lf, col_name):
    """
    device_0_device_name 컬럼의 특수문자 제거 및 소문자 변환
    """
    return maude_lf.with_columns(
        pl.col(col_name)
        .str.replace_all(r'[^a-zA-Z0-9\s]', '')  # 특수문자 제거
        .str.to_lowercase()  # 소문자 변환
        .str.strip_chars()  # 앞뒤 공백 제거
        .str.replace_all(r'\s+', ' ')  # 연속된 공백을 하나로
        .alias(col_name)
    )
    
def device_name_product_code_match(maude_lf, name_col, code_col):
    """
    device_0_device_name과 device_0_device_product_code를 매칭시켜서
    한 product_code에 여러 device_name이 있는 경우 가장 빈도가 높은 이름으로 통일
    """ 
    # product_code별로 가장 빈도가 높은 device_name 선택
    code_to_name = (
        maude_lf
        .select([name_col, code_col])
        .filter(pl.col(name_col).is_not_null() & pl.col(code_col).is_not_null())
        .group_by([code_col, name_col])  # 먼저 그룹화해서 빈도 계산
        .count()
        .sort("count", descending=True)
        .group_by(code_col)
        .first()  # 각 code별 가장 빈도 높은 이름 선택
        .select([code_col, pl.col(name_col).alias("canonical_name")])
    )
    
    # 원본 데이터에 매핑 테이블 조인
    result = (
        maude_lf
        .join(
            code_to_name,
            on=code_col,
            how="left"
        ) 
        .with_columns(
            pl.coalesce(pl.col("canonical_name"), pl.col(name_col)).alias(name_col)
        )
        .drop("canonical_name")
    )
    
    return result

In [11]:
# # 테스트
# # 1. device_name 클린징
# cleaned = device_name_clean(df.head(10000), "device_0_openfda_device_name")

# # 2. product_code와 매칭
# result = device_name_product_code_match(cleaned, "device_0_openfda_device_name", "device_0_device_report_product_code")

# # 확인
# result.select(['device_0_openfda_device_name','device_0_device_report_product_code']).collect()

### manufacturer_d_name & postal_code 
- 매칭 후 join (최빈값)

In [12]:
def manufacturer_postal_match(maude_lf, name_col, postal_col):
    """
    manufacturer_d_name을 manufacturer_d_postal_code로 매칭시켜서
    이름 없는 것들 채우고 있는 것들 통일하기
    """
    # 우편번호별로 가장 빈도가 높은 이름 선택
    postal_to_name = (
        maude_lf
        .select([name_col, postal_col])
        .filter(pl.col(name_col).is_not_null() & pl.col(postal_col).is_not_null())
        .unique()
        .group_by(postal_col)
        .agg(
            pl.col(name_col).mode().first().alias("canonical_name")
        )
    )
    
    # 원본 데이터에 매핑 테이블 조인
    result = (
        maude_lf
        .join(
            postal_to_name,
            on=postal_col,
            how="left"
        )
        .with_columns(
            # canonical_name이 있으면 사용, 없으면 원래 이름 유지
            pl.coalesce(pl.col("canonical_name"), pl.col(name_col)).alias(name_col)
        )
        .drop("canonical_name")
    )
    
    return result

In [13]:
match_test_result = manufacturer_postal_match(maude_lf.head(100000), 'device_0_manufacturer_d_name', 'device_0_manufacturer_d_postal_code').collect()

match_test_result.select(['device_0_manufacturer_d_name', 'device_0_manufacturer_d_postal_code'])

device_0_manufacturer_d_name,device_0_manufacturer_d_postal_code
str,str
"""MPRI""","""00766"""
"""ALLERGAN (COSTA RICA)""",null
"""VALENCIA TECHNOLOGIES CORP""","""91355"""
"""W. L. GORE & ASSOCIATES, INC.""","""86004"""
"""MEDTRONIC PUERTO RICO OPERATIO…","""00777-3869"""
…,…
"""MEDTRONIC NEUROMODULATION""","""55432"""
"""MEDTRONIC MINIMED""","""91325"""
"""HEARTWARE, INC.""","""55112"""


### device_report_product_code 특문 시작 코드 삭제

In [14]:
def product_code_clean(maude_lf, col_name):
    """
    device_0_device_product_code 컬럼의 영어 대문자 이외 문자 제거
    """
    return maude_lf.with_columns(
        pl.col(col_name)
        .str.replace_all(r'[^A-Z]', '')  # 영어 대문자 이외 문자 제거
        .alias(col_name)
    )

### 나이 컷

In [15]:
def age_cut(maude_lf, col_name):
    """
    age col이 들어오면 0~120으로 범위를 한정해주는 함수.
    0 이하는 0으로, 120 이상은 120으로 limit
    """
    return maude_lf.with_columns(
        pl.col(col_name)
        .str.replace("NA", "")  # "NA"를 빈 문자열로 (null 처리)
        .str.extract(r"(\d+)", 1)  # 숫자만 추출
        .cast(pl.Float64)  # 숫자로 변환
        .clip(0, 120)  # 0~120 범위로 제한
        .alias(col_name)
    )

In [16]:
age_limited = age_cut(maude_lf, "patient_0_patient_age")
age_limited.select(
    pl.col("patient_0_patient_age").min().alias("min_age"),
    pl.col("patient_0_patient_age").max().alias("max_age")
).collect()

min_age,max_age
f64,f64
0.0,120.0


### 1900년대 -> 규리님 year 변환 필요

In [17]:
maude_lf.collect_schema()["date_of_event"]


String

In [18]:
# 1) clean 컬럼 생성 (dtype 고정)
scan_preprocess = maude_lf.with_columns(
    pl.when(pl.col("date_of_event").dt.year().is_between(1900, 1999))
      .then(pl.lit(None, dtype=pl.Date))          # <- 핵심: Date로 null 지정
      .otherwise(pl.col("date_of_event"))        
      .alias("date_of_event_clean")
)

### 팀장님꺼

In [19]:
# # 1단계
# preprocessor1 = create_udi_preprocessor()
# preprocessor1.apply_to_lazyframe(
#     maude_lf, 'device_0_udi_di', maude_step1_path, chunk_size=10_000
# )
# del preprocessor1  # 명시적 삭제

In [20]:
# # 2단계
# maude_lf2 = pl.scan_parquet(maude_step1_path)
# preprocessor2 = create_company_preprocessor()
# preprocessor2.apply_to_lazyframe(
#     maude_lf2, ['device_0_manufacturer_d_name', 'device_0_brand_name'], 
#     maude_step2_path, chunk_size=10_000
# )
# del maude_lf2, preprocessor2

In [21]:
# # 중간 파일 정리
# maude_step1_path.unlink(missing_ok=True)

# # 3단계
# maude_lf3 = pl.scan_parquet(maude_step2_path)
# preprocessor3 = create_generic_preprocessor()
# preprocessor3.apply_to_lazyframe(
#     maude_lf3, ['device_0_model_number', 'device_0_catalog_number', 'device_0_lot_number'], 
#     maude_step3_path, chunk_size=10_000
# )
# del maude_lf3, preprocessor3

In [22]:
maude_step2_path.unlink(missing_ok=True)

# maude 데이터 불러오기
loader3 = DataLoader(
    start=2024,
    end=2024,
    output_file = maude_step3_path,
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
    # 'n_rows': 1000
}
cleaned_maude_lf = loader3.load(adapter=adapter, **polars_kwargs)
cleaned_maude_lf


📖 /Users/t2023-m0030/Desktop/i_hate_maude/data/silver/clean_step3.parquet 로딩 중... (adapter=polars)


# udi 매칭

### primary 컬럼 생성

In [23]:
# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=DATA_DIR/'udi.parquet',
    max_workers = 2
)

udi_lf = udi_loader.load(adapter, **polars_kwargs)
udi_lf


📖 /Users/t2023-m0030/Desktop/i_hate_maude/data/udi.parquet 로딩 중... (adapter=polars)


In [24]:
# udi_loader.process()

클렌징 전 필터링 (class 3)

In [25]:
import polars.selectors as cs

# 방법 1: regex 패턴에 매칭되는 컬럼 중 하나라도 3을 포함하는 행 필터링
udi_lf = udi_lf.filter(
    pl.any_horizontal(cs.matches('*device_class$') == '3')
)

udi_lf.select(pl.len()).collect()

len
u32
72460


In [26]:
drop_patterns = CUSTOMER_PATTERNS + DEVICE_SIZE_PATTERNS + STORAGE_PATTERNS

regex = "|".join(drop_patterns)

udi_lf = udi_lf.select(
    ~cs.matches(regex)
)

# udi_lf.collect_schema().names()

In [27]:
udi_di_cols = get_pattern_cols(udi_lf, UDI_DI_PATTERNS)
identifiers_cols = get_pattern_cols(udi_lf, IDENTIFIER_PATTERNS)
type_cols = get_pattern_cols(udi_lf, TYPE_PATTERNS)

In [28]:
# type-udi_di 쌍 만들기 (인덱스로 매칭)
def extract_index(col_name):
    match = re.search(r'identifiers_(\d+)_', col_name)
    return int(match.group(1)) if match else None

type_id_pairs = []
for type_col in type_cols:
    idx = extract_index(type_col)
    udi_di_col = f'identifiers_{idx}_id'
    if udi_di_col in udi_di_cols:
        type_id_pairs.append((type_col, udi_di_col))

len(type_id_pairs)

132

In [29]:
primary_udi_unique = set()

for type_col, id_col in tqdm(type_id_pairs, desc="Processing columns", unit="col"):
    try:
        count = (
            udi_lf
            .filter(pl.col(type_col).eq("Primary"))
            .select(pl.len())
            .collect()
            .item()
        )
        
        if count > 0:
            ids = (
                udi_lf
                .filter(pl.col(type_col).eq("Primary"))
                .select(pl.col(id_col))
                .unique()
                .collect()
                .to_series()
                .drop_nulls()
                .to_list()
            )
            primary_udi_unique.update(ids)
            
    except Exception as e:
        tqdm.write(f"Error processing {type_col}: {e}")
        continue

print(f"\n{'='*50}")
print(f"UDI 데이터의 고유 Primary udi 개수: {len(primary_udi_unique):,}")
print(f"{'='*50}")

Processing columns: 100%|██████████| 132/132 [00:03<00:00, 34.27col/s]


UDI 데이터의 고유 Primary udi 개수: 72,297


In [30]:
udi_step1_path = DATA_DIR / 'silver' / 'udi_primary.parquet'
udi_step2_path = DATA_DIR / 'silver' / 'udi_clean.parquet'

## 매칭 시작

In [31]:
# # Primary인 id를 추출
# def primary_transform(lf: pl.LazyFrame):
#     return lf.with_columns(
#         pl.coalesce([
#             pl.when(pl.col(type_col).eq("Primary"))
#             .then(pl.col(id_col))
#             for type_col, id_col in type_id_pairs
#         ]).alias('primary_udi_di')
#     )

# process_lazyframe_in_chunks(
#     udi_lf, 
#     primary_transform, 
#     udi_step1_path, 
#     10_000,
#     desc="Primary extraction"
# )

In [32]:
# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=udi_step1_path,
)

primary_udi_lf = udi_loader.load(adapter, **polars_kwargs)

❌ 파일이 존재하지 않습니다: /Users/t2023-m0030/Desktop/i_hate_maude/data/silver/udi_primary.parquet


In [33]:
# # 회사 이름 정제
# preprocessor1 = create_company_preprocessor()

# preprocessor1.apply_to_lazyframe(
#     primary_udi_lf,
#     ['company_name', 'brand_name'],
#     udi_step2_path,
#     10_000,
# )

# del preprocessor1, primary_udi_lf
# udi_step1_path.unlink(missing_ok=True)

In [34]:
# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=udi_step2_path,
)

cleaned_udi_lf = udi_loader.load(adapter, **polars_kwargs)


📖 /Users/t2023-m0030/Desktop/i_hate_maude/data/silver/udi_clean.parquet 로딩 중... (adapter=polars)


In [35]:
rename_udi_lf = cleaned_udi_lf.rename({
    'company_name': 'manufacturer',
    'brand_name': 'brand',
    'version_or_model_number': 'model_number',
    'primary_udi_di': 'udi_di',
})

rename_maude_lf = cleaned_maude_lf.rename({
    'device_0_manufacturer_d_name': 'manufacturer',
    'device_0_brand_name': 'brand',
    'device_0_model_number': 'model_number',
    'device_0_catalog_number': 'catalog_number',
    'device_0_lot_number': 'lot_number',
    'device_0_udi_di': 'udi_di',
    'device_0_udi_public': 'udi_public'
})

In [36]:
target_cols = [
    'manufacturer',
    'brand',
    'model_number',
    'catalog_number'
]

join_col = 'udi_di'

common_cols = target_cols + [join_col]

maude_cols = common_cols + [
    'mdr_report_key',
]

udi_cols = common_cols + udi_di_cols

In [37]:
udi_necessary_lf = rename_udi_lf.select(pl.col(udi_cols))
maude_necessary_lf = rename_maude_lf.select(pl.col(maude_cols))

### 고유값 

In [38]:
cols_group = {
    'udi': udi_di_cols,
}

# udi 데이터셋의 udi_di 고유값
udi_udi_unique = get_unique_by_cols_safe(
    udi_necessary_lf, 
    cols_group,
    memory_safety_ratio=0.3,
    calibration_factor = 1
)['udi']

문자열 길이 통계를 실제 데이터로부터 추정 중...
  문자열 길이 통계:
    - 평균(mean): 14.1자
    - 중앙값(median): 14.0자
    - 75%ile: 14.0자
    - 90%ile: 14.0자
    - 최댓값: 19자
  → 사용할 크기(p75): 14.0자

=== 메모리 기반 자동 임계값 계산 ===
사용 가능한 메모리: 3.11 GB
안전 사용 메모리 (30%): 0.93 GB
예상 바이트/고유값: 106 bytes
  - 문자열 데이터: 28 bytes
  - str 오버헤드: 50 bytes
  - set 오버헤드: 28 bytes
계산된 최대 고유값: 9,449,147개



Extracting unique values: 100%|██████████| 1/1 [00:00<00:00,  3.90it/s]

udi: 75,073개의 고유값 (예상 메모리: 7.6 MB)
  ✓ udi 추출 완료 (실제 메모리: 6.52 MB)
    → 예상치 정확도: 116.5% (예상/실제 비율)


=== 추출 요약 ===
성공: 1/1
실패/스킵: 0/1


In [39]:
maude_udi_unique = get_unique(maude_necessary_lf, ['udi_di'])
angry_udi_unique = maude_udi_unique - udi_udi_unique
survive_udi_unique = maude_udi_unique & udi_udi_unique
maude_primary_udi_unique = survive_udi_unique & primary_udi_unique
maude_secondary_udi_unique = survive_udi_unique - primary_udi_unique

In [40]:
class UniqueUDIDI:
    def __init__(
        self,
        udi_udi_unique: set = None,
        maude_udi_unique: set = None,
        angry_udi_unique: set = None,
        survive_udi_unique: set = None,
        primary_udi_unique: set = None,
        maude_primary_udi_unique: set = None,
        maude_secondary_udi_unique: set = None
    ):
        self.udi = udi_udi_unique
        self.maude = maude_udi_unique
        self.angry = angry_udi_unique
        self.survive = survive_udi_unique
        self.primary = primary_udi_unique
        self.maude_primary = maude_primary_udi_unique
        self.maude_secondary = maude_secondary_udi_unique

    def print_stats(self):
        print(f'UDI 데이터의 고유 udi 개수: {len(self.udi)}개')
        print(f'UDI 데이터의 고유 Primary udi 개수: {len(self.primary)}개')
        print(f'MAUDE 데이터의 고유 udi 개수: {len(self.maude)}개')
        print(f'UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: {len(self.survive)}개')
        print(f'Primary UDI인 MAUDE 데이터의 고유 udi 개수: {len(self.maude_primary)}개')
        print(f'Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: {len(self.maude_secondary)}개')
        print(f'UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: {len(self.angry)}개')

unique_udi_di = UniqueUDIDI(udi_udi_unique, maude_udi_unique, angry_udi_unique, survive_udi_unique, primary_udi_unique, maude_primary_udi_unique, maude_secondary_udi_unique)

unique_udi_di.print_stats()

UDI 데이터의 고유 udi 개수: 75072개
UDI 데이터의 고유 Primary udi 개수: 72297개
MAUDE 데이터의 고유 udi 개수: 12504개
UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: 7974개
Primary UDI인 MAUDE 데이터의 고유 udi 개수: 7950개
Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: 24개
UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: 4530개


### primary와 매칭해서 가져오는 함수

In [41]:
def extract_from_match(
    src_lf: pl.LazyFrame, desc_lf: pl.LazyFrame, 
    on: str | Sequence[str], 
    target_cols: str | Sequence[str]
):
    if isinstance(on, str):
        on = [on]
    
    if isinstance(target_cols, str):
        target_cols = [target_cols]
    
    udi_subset = src_lf.select(pl.col(on + target_cols))
    
    result = desc_lf.join(
        udi_subset,
        on=on,
        how='left'
    )
    
    for col in target_cols:
        result = result.with_columns(
            pl.coalesce([f'{col}_right', col]).alias(col)
        ).drop(f'{col}_right')
    
    return result

maude_match_lf = extract_from_match(udi_necessary_lf, rename_maude_lf, join_col, target_cols)

# cleaned_maude_lf.head(10).collect().to_pandas()
maude_match_lf.head(10).collect().to_pandas()

,mdr_report_key,report_number,adverse_event_flag,product_problem_flag,date_of_event,date_received,device_date_of_manufacturer,event_type,previous_use_code,single_use_flag,...,mdr_text_5_text,mdr_text_5_text_type_code,mdr_text_6_text,mdr_text_6_text_type_code,mdr_text_7_text,mdr_text_7_text_type_code,mdr_text_8_text,mdr_text_8_text_type_code,mdr_text_9_text,mdr_text_9_text_type_code
0,19134527,2649622-2024-10780,Y,Y,20240410,20240418,20230113,Injury,I,Y,...,None,None,None,None,None,None,None,None,None,None
1,19234872,9617229-2024-07636,Y,Y,20240311,20240502,20160905,Injury,I,Y,...,HEALTHCARE PROFESSIONAL REPORTED LEFT SIDE DEF...,Description of Event or Problem,DEVICE EVALUATION: BASED ON THE PRODUCT ANALYS...,Additional Manufacturer Narrative,"HEALTHCARE PROFESSIONAL REPORTED, LEFT SIDE DE...",Description of Event or Problem,HEALTHCARE PROFESSIONAL REPORTED LEFT SIDE DEF...,Description of Event or Problem,None,None
2,19473107,3006630150-2024-03625,Y,Y,20240226,20240605,20230627,Injury,I,Y,...,None,None,None,None,None,None,None,None,None,None
3,19474199,2017233-2024-05020,N,Y,20240510,20240606,20231109,Malfunction,I,Y,...,None,None,None,None,None,None,None,None,None,None
4,19555551,2032227-2024-187624,N,Y,20240528,20240618,20211102,Malfunction,A,N,...,None,None,None,None,None,None,None,None,None,None
5,19565729,2032227-2024-188679,N,Y,20240531,20240619,20230524,Malfunction,A,N,...,None,None,None,None,None,None,None,None,None,None
6,19631462,2649622-2024-17126,N,Y,20240607,20240628,20240222,Malfunction,I,Y,...,None,None,None,None,None,None,None,None,None,None
7,19345151,2649622-2024-13681,Y,Y,20240425,20240517,20090720,Injury,I,Y,...,IT WAS FURTHER REPORTED THAT THE RV LEAD EXHIB...,Description of Event or Problem,MEDTRONIC SUBMITS THIS REPORT TO COMPLY WITH F...,Additional Manufacturer Narrative,None,None,None,None,None,None
8,19467176,1220648-2024-12017,Y,N,20240526,20240605,20240508,Death,I,Y,...,None,None,None,None,None,None,None,None,None,None
9,19471902,2649622-2024-15071,Y,Y,20240514,20240605,20140423,Injury,I,Y,...,IT WAS FURTHER REPORTED THAT THE RV LEAD WAS C...,Description of Event or Problem,None,None,None,None,None,None,None,None


In [42]:
analyze_null_values(maude_match_lf, common_cols)
analyze_null_values(rename_maude_lf, common_cols)
overview_col(maude_match_lf, 'brand', n_rows=100)
overview_col(rename_maude_lf, 'brand', n_rows=100)
overview_col(maude_lf, 'device_0_brand_name', n_rows=1000)


=== 결측치 분석 ===
전체 행 수: 566,454

udi_di                                       :    246,605개 ( 43.53%)
catalog_number                               :    120,814개 ( 21.33%)
model_number                                 :     82,268개 ( 14.52%)
brand                                        :     12,063개 (  2.13%)
manufacturer                                 :        473개 (  0.08%)

=== 결측치 분석 ===
전체 행 수: 565,168

udi_di                                       :    246,605개 ( 43.63%)
catalog_number                               :    173,724개 ( 30.74%)
model_number                                 :    162,441개 ( 28.74%)
brand                                        :     12,199개 (  2.16%)
manufacturer                                 :        554개 (  0.10%)
brand의 고유 개수: 6418


,head_brand,tail_brand
0,None,XMTR GST4C MMT7821WW JP NONLOCKOUT TSTD
1,1366 LEAD VDDR,XMTR KIT MMT7840W1 GRDN 4 9L W
2,14F MANTA VASCULAR CLOSURE DEVICE,XMTR KIT MMT7840W2 GRDN 4 9L W2
3,150,XMTR KIT MMT7840W3 GRDN 4 7L W3
4,151,XMTR KIT MMT7840W6 GRDN 4 9L W6
...,...,...
95,640G INSULIN PUMP MMT1712X,ZOLL X SERIES DEFIBRILLATOR
96,670G INSULIN PUMP,ZOLL X SERIES DEFIBRILLATORMONITOR TEMPERATURE...
97,670G INSULIN PUMP MMT1780KL,ZOLL XSERIES ADVANCED
98,680G INSULIN PUMP,ZOOM LATITUDE


brand의 고유 개수: 6820


,head_brand,tail_brand
0,None,XIENCE XPEDITION
1,1 NBCA TA OIL & ACCESSORIES,XIENCE XPEDITION 48 EVEROLIMUS ELUTING CORONAR...
2,1366 LEAD VDDR,XIENCE XPEDITION EVEROLIMUS ELUTING CORONARY S...
3,1410Z KENDALL 1PRPK 10PRCA,XMTR GST3B WW V30A TESTED
4,14F MANTA,XMTR GST4C MMT7821LWW LOCKOUT TESTED
...,...,...
95,630G INSULIN PUMP MMT1715KL 630G,ZOLL X SERIES DEFIBRILLATOR
96,630G INSULIN PUMP MMT1715KR 630G,ZOLL X SERIES DEFIBRILLATORMONITOR TEMPERATURE...
97,640G INSULIN PUMP DEMMT1711K 640G BLACK MM,ZOLL XSERIES ADVANCED
98,640G INSULIN PUMP DEMMT1712K 640G BLACK MG,ZOOM LATITUDE


device_0_brand_name의 고유 개수: 7360


,head_device_0_brand_name,tail_device_0_brand_name
0,None,TAPER SLEEVE ADAPTER 12/14 +2
1,"""GORE- VIATORRI-TIPS ENDOPROSTHESIS WITH CONTR...",TAPER SLEEVE ADAPTER 12/14 +8
2,-S THERMOCOOL® SMART TOUCH® SF UNI-DIRECTIONAL...,TAPERLOC 12/14 POR LAT 11X142
3,"1 NBCA, TA, OIL, & ACCESSORIES",TAPERLOC 133 MP SP T1 PPS HO 8X101
4,"1.5T LINX, 13B",TAPERLOC 133 MP SP T1 PPSHO 6X97.5
...,...,...
995,"BLOOD PUMP PU VALVES 15 ML, IN/OUT Ø9 MM",ZOLL X SERIES DEFIBRILLATOR
996,"BLOOD PUMP PU VALVES 25 ML, IN/OUT Ø9 MM",ZOLL X SERIES DEFIBRILLATOR/MONITOR TEMPERATUR...
997,"BLOOD PUMP PU VALVES 30 ML, IN/OUT Ø9 MM",ZOLL X-SERIES ADVANCED
998,"BLOOD PUMP PU VALVES 50 ML, IN/OUT Ø12 MM",ZOOM LATITUDE


In [43]:
import polars as pl
from pathlib import Path
from src.preprocess.preprocessor import UDIProcessor
from src.preprocess.config import Config


output_path=DATA_DIR / "maude_with_udi.parquet"

# # 처리
# processor = UDIProcessor(Config())
# result_path = processor.process(
#     maude_lf=rename_maude_lf,
#     udi_lf=rename_udi_lf,
#     output_path=Path(output_path),
#     chunk_size=10_000
# )


In [44]:
loader4 = DataLoader(
    name='event',
    output_file=output_path
)

matched_lf = loader4.load(adapter=adapter, **polars_kwargs)


📖 /Users/t2023-m0030/Desktop/i_hate_maude/data/maude_with_udi.parquet 로딩 중... (adapter=polars)


# 중복 제거

In [45]:
matched_lf.collect_schema().names()

['mdr_report_key',
 'report_number',
 'adverse_event_flag',
 'product_problem_flag',
 'date_of_event',
 'date_received',
 'device_date_of_manufacturer',
 'event_type',
 'previous_use_code',
 'single_use_flag',
 'report_source_code',
 'reprocessed_and_reused_flag',
 'report_to_fda',
 'event_location',
 'manufacturer_link_flag',
 'pma_pmn_number',
 'manufacturer',
 'device_0_manufacturer_d_postal_code',
 'brand',
 'catalog_number',
 'model_number',
 'udi_di',
 'lot_number',
 'udi_public',
 'device_0_device_report_product_code',
 'device_0_device_operator',
 'device_0_openfda_device_class',
 'device_0_openfda_device_name',
 'patient_0_patient_age',
 'patient_0_patient_sex',
 'patient_0_patient_weight',
 'patient_0_patient_race',
 'patient_0_patient_problems',
 'patient_0_sequence_number_outcome',
 'patient_0_sequence_number_treatment',
 'mdr_text_0_text',
 'mdr_text_0_text_type_code',
 'mdr_text_10_text',
 'mdr_text_10_text_type_code',
 'mdr_text_11_text',
 'mdr_text_11_text_type_code',
 

In [46]:
# 한 번에 처리
columns_with_final = [col for col in matched_lf.collect_schema().names() if '_final' in col]

matched_lf = matched_lf.with_columns([
    pl.col(col).alias(col.replace('_final', ''))
    for col in columns_with_final
]).drop(columns_with_final)

In [47]:
# 1차 : 중복 제거 컬럼 
dedup_cols = [
    'report_number',
    'date_of_event', 
    'manufacturer',
    'device_signature',
    'lot_number',
    'udi_public'
]

# Unknown / N/A 패턴 리스트
na_patterns = r'^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$'



In [48]:
# 일단 중복된 행들만 확인
# 조합 (6개 컬럼)의 개수
matched_lf_with_cnt = matched_lf.with_columns(
    pl.len().over(dedup_cols).alias('duplicate_cnt')
)

# cnt가 2 이상이 경우에만 
# cnt가 1인 경우에는 삭제할 필요가 없으므로
matched_lf_duplicates_only = matched_lf_with_cnt.filter(
    pl.col('duplicate_cnt') >= 2
)

duplicate_cnt = matched_lf_duplicates_only.select(pl.len()).collect().item()
print(f"중복된 행의 개수: {duplicate_cnt:,}개")

unique_cnt = matched_lf.unique(subset=dedup_cols, maintain_order=True).select(pl.len()).collect().item()
print(f"유일한 행의 개수: {unique_cnt:,}개")

중복된 행의 개수: 6,314개
유일한 행의 개수: 571,282개


In [49]:
def remove_na_values(matched_lf, dedup_cols, na_patterns, verbose=True):
    """
    Na / Unknown 값이 있는 행을 제거하는 함수
    
    작동방식:
    1. 각 컬럼에 대해 유효한 값인지 체크
    2. 모든 조건을 포함한(모두 만족하는) 행으로 필터링
    3. 필터 적용

    Parameters:
    -----------
    matched_lf : polars.LazyFrame
        원본 LazyFrame
    dedup_cols : list
        체크할 컬럼 리스트
    na_patterns : str
        NA / Unknown 패턴 정규식
    verbose : bool
        진행상황 출력 여부

    Returns:
    --------
    polars.LazyFrame
        NA 값이 제거된 LazyFrame
    """
    
    # 진행상황 확인
    if verbose:
        print("NA 값 제거")
        print(f"패턴: {na_patterns}")
    
    # 제거되기 전 개수 확인
    before_cnt = matched_lf.select(pl.len()).collect().item()
    if verbose:
        print(f"제거 전 행 개수: {before_cnt:,}개")
    
    # 각 컬럼별로 필터 조건
    conditions = []
    
    for col in dedup_cols:
        # 컬럼이 존재하는지 확인
        if col in matched_lf.collect_schema().names():
            # 유효한 값의 조건
            # null이 아니고 na_patterns 패턴에 매칭되지 않는 값
            cond = (
                pl.col(col).is_not_null()
                & ~pl.col(col).cast(pl.Utf8).str.to_uppercase().str.contains(na_patterns)
            )
            conditions.append(cond)
            
            if verbose:
                print(f"  컬럼 '{col}'에 대해 NA/Unknown 값 제거 조건 추가")
        else:
            if verbose:
                print(f"  컬럼 '{col}'이(가) 존재하지 않음. 건너뜀")
    
    # 예외처리
    if not conditions:
        if verbose:
            print("제거할 조건이 없음. 원본 반환")
        return matched_lf
    
    # 모든 조건을 AND 조건으로 결합
    final_condition = conditions[0]
    for cond in conditions[1:]:
        final_condition = final_condition & cond
    
    # 필터 적용
    matched_lf_cleaned = matched_lf.filter(final_condition)
    
    # 결과
    after_cnt = matched_lf_cleaned.select(pl.len()).collect().item()
    removed_cnt = before_cnt - after_cnt
    
    if verbose:
        print(f"제거 후 행 개수: {after_cnt:,}개")
        print(f"제거된 행 개수: {removed_cnt:,}개")
    
    return matched_lf_cleaned

In [50]:
def analyze_duplicates(matched_lf, group_cols, verbose = True):
    """
    중복 데이터 분석 함수들

    작동 방식 
    1. 전체 개수 확인
    2. 고유(unique) 개수 확인
    3. 전체 - 고유 = 중복 개수 확인

    Parameters:
    matched_lf : polars.DataFrame -> 원본
    dedup_cols : list -> 중복 확인 컬럼
    verbose : bool -> 진행상황 출력 여부

    returns:

    tuple : (전체 개수, 고유 개수, 중복 개수)
    """

    if verbose:
        print(f" 중복 확인")

    # 전체 개수
    total_cnt = matched_lf.select(pl.len()).collect().item()

    # 고유 개수
    unique_cnt = matched_lf.unique(
        subset = group_cols, # 중복 ㅎ판단
        maintain_order = True
    ).select(pl.len()).collect().item()

    # 중복 개수
    duplicate_cnt = total_cnt - unique_cnt

    if verbose:
        print(f"전체 개수 : {total_cnt:,}개")
        print(f"고유 개수 : {unique_cnt:,}개")
        print(f"중복 개수 : {duplicate_cnt:,}개")
        for i, col in enumerate(dedup_cols, start=1):
            print(f"{i}. {col}")

    return total_cnt, unique_cnt, duplicate_cnt

In [51]:
def remove_duplicates(matched_lf, dedup_cols, keep = 'first', verbose = True):
    """
    중복 데이터 제거 함수

    작동 방식
    1. dedup_cols 기준으로 중복 판단
    2. keep 옵션에 따라 첫번째/마지막 행 유지
    3. 중복 제거된 DF 반환

    Parameters:
    matched_lf : polars.DataFrame -> 원본 DF
    dedup_cols : list -> 중복 판단 컬럼 리스트
    keep : str -> 'first' or 'last'
        'first' : 첫번째 행 유지
        'last' : 마지막 행 유지
    verbose : bool -> 진행상황 출력 여부

    Returns:
    polars.DataFrame -> 중복 제거된 DF
    """

    if verbose:
        print("중복 제거 시작")
        print(f"중복 판단 컬럼: {dedup_cols}")
        print(f"유지 옵션: {keep}")

    # 중복 제거
    matched_lf_deduped = matched_lf.unique(
        subset = dedup_cols,
        maintain_order = True,
        keep = 'first'
    )

    if verbose:
        before_cnt = matched_lf.select(pl.len()).collect().item()
        after_cnt = matched_lf_deduped.select(pl.len()).collect().item()
        removed_cnt = before_cnt - after_cnt

        print(f"제거 전 행 개수 : {before_cnt:,}개")
        print(f"제거 후 행 개수 : {after_cnt:,}개")
        print(f"제거된 행 개수 : {removed_cnt:,}개")

    return matched_lf_deduped

In [52]:
# na 값들 제거
matched_lf_cleaned = remove_na_values(matched_lf_duplicates_only, dedup_cols, na_patterns)
print(matched_lf_cleaned)

NA 값 제거
패턴: ^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$
제거 전 행 개수: 6,314개
  컬럼 'report_number'에 대해 NA/Unknown 값 제거 조건 추가
  컬럼 'date_of_event'에 대해 NA/Unknown 값 제거 조건 추가
  컬럼 'manufacturer'에 대해 NA/Unknown 값 제거 조건 추가
  컬럼 'device_signature'에 대해 NA/Unknown 값 제거 조건 추가
  컬럼 'lot_number'에 대해 NA/Unknown 값 제거 조건 추가
  컬럼 'udi_public'에 대해 NA/Unknown 값 제거 조건 추가
제거 후 행 개수: 10개
제거된 행 개수: 6,304개
naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

FILTER [([([([([(col("udi_public").str.uppercase().str.contains(["^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$"]).not()) & (col("report_number").str.uppercase().str.contains(["^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$"]).not())]) & (col("date_of_event").str.uppercase().str.contains(["^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$"]).not())]) & (col("manufacturer").str.uppercase().str.cont

In [53]:
total, unique, duplicate = analyze_duplicates(matched_lf_cleaned, dedup_cols)
pprint((total, unique, duplicate))  

 중복 확인
전체 개수 : 10개
고유 개수 : 5개
중복 개수 : 5개
1. report_number
2. date_of_event
3. manufacturer
4. device_signature
5. lot_number
6. udi_public
(10, 5, 5)


In [54]:
matched_lf_final = remove_duplicates(matched_lf_cleaned, dedup_cols, keep='first')
print(matched_lf_final)

중복 제거 시작
중복 판단 컬럼: ['report_number', 'date_of_event', 'manufacturer', 'device_signature', 'lot_number', 'udi_public']
유지 옵션: first
제거 전 행 개수 : 10개
제거 후 행 개수 : 5개
제거된 행 개수 : 5개
naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

UNIQUE[maintain_order: true, keep_strategy: First] BY Some(["report_number", "date_of_event", "manufacturer", "device_signature", "lot_number", "udi_public"])
  FILTER [([([([([(col("udi_public").str.uppercase().str.contains(["^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$"]).not()) & (col("report_number").str.uppercase().str.contains(["^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$"]).not())]) & (col("date_of_event").str.uppercase().str.contains(["^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$"]).not())]) & (col("manufacturer").str.uppercase().str.contains(["^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NU

# 저품질 행, 필터링

- udi 매칭이 안된 것들 + 매칭하고 품질이 낮은 것들(udi-di , 제조업체 null 값 등) + mdr_text(품질이 별루..)

 'udi_confidence' 이 친구가 아주아주 나쁨 별루,...

제조업체, 제품군, 클래스, mdr_text, event_type

# 타입 변환

## Categorical

### device_class
- categorical
- 우리는 device 3만 뽑은 상태
- 그렇기 때문에 변환할 필요가 있을까? (일단 주석 처리)

In [55]:
# def nullify_non_123_openfda_device_class(lf: pl.LazyFrame) -> pl.LazyFrame:
#     """
#     이름이 '*_openfda_device_class'로 끝나는 모든 컬럼을 찾아,
#     값이 "1", "2", "3"인 경우만 유지하고 나머지는 null로 치환한다.

#     - 문자열 양끝 공백은 제거(strip)해서 비교한다.
#     - 원본이 categorical이어도 비교는 문자열로 안전하게 처리한다.
#     - 컬럼이 없으면 입력 LazyFrame을 그대로 반환한다.
#     """
#     schema_cols = lf.collect_schema().names()
#     target_cols = [c for c in schema_cols if c.endswith("_openfda_device_class")]

#     if not target_cols:
#         return lf

#     allowed = ["1", "2", "3"]

#     exprs = []
#     for c in target_cols:
#         cleaned = pl.col(c).cast(pl.Utf8).str.strip_chars()

#         exprs.append(
#             pl.when(cleaned.is_in(allowed))
#               .then(cleaned)          # "1","2","3"만 남김
#               .otherwise(None)        # 나머지는 null
#               .alias(c)               # 원본 컬럼 덮어쓰기
#         )

#     return lf.with_columns(exprs)

In [56]:
# def cast_device_openfda_classes_to_cat(matched_lf: pl.LazyFrame) -> pl.LazyFrame:
#     """
#     이름이 '*_openfda_device_class'로 끝나는 모든 컬럼을 Categorical로 변환
#     예: device_0_openfda_device_class, device_1_openfda_device_class, ...
#     """
#     # 1) 컬럼 이름 중에서 패턴 맞는 것만 골라오기
#     target_cols = [
#         col for col in matched_lf.columns
#         if col.endswith("_openfda_device_class")
#     ]

#     if not target_cols:
#         return matched_lf  # 해당 컬럼이 없으면 그대로 반환

#     # 2) 골라낸 컬럼들을 전부 Categorical로 캐스팅
#     exprs = [pl.col(col).cast(pl.Categorical) for col in target_cols]

#     return matched_lf.with_columns(exprs)

### report_source_code
- categorical

In [57]:
def convert_report_source_code(matched_lf, verbose=True):
    """
    report_source_code를 Categorical 타입으로 변환
    
    변환 이유:
    - report_source_code는 제한된 카테고리 값만 가짐
    - Categorical로 변환하면 메모리 절약됨
    
    Parameters:
    -----------
    matched_lf : polars.LazyFrame
        입력 데이터
    verbose : bool
        값 확인 결과 출력 여부
    
    Returns:
    --------
    polars.LazyFrame
        변환된 데이터
    """
    # 값 확인
    if verbose:
        print("\nreport_source_code 값 확인:")
        value_counts = matched_lf.select(
            pl.col('report_source_code').value_counts().sort('count', descending=True)
        ).collect()
        print(value_counts)
        print(f"\n고유값 개수: {value_counts.height}개")
    
    # Categorical 타입으로 변환
    return matched_lf.with_columns(
        pl.col('report_source_code').cast(pl.Categorical)
    )

### date_location
- categorical

In [58]:
# CATEGORICAL 타입 변환 함수
def cast_to_categorical(matched_lf: pl.LazyFrame, col: str) -> pl.LazyFrame:
    """
    단일 컬럼을 Categorical로 변환
    """
    if col not in matched_lf.columns:
        print(f"[WARN] Column '{col}' not found. Skipped.")
        return matched_lf


    return matched_lf.with_columns(
        pl.col(col).cast(pl.Categorical)
    )

scan_preprocess = cast_to_categorical(scan_preprocess, "event_location")

/var/folders/qk/10b0q2j971j6d2py2x39ys200000gn/T/ipykernel_40667/1705860562.py:6: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if col not in matched_lf.columns:


### event_type
- categorical

In [59]:
# CATEGORICAL 타입 변환 함수
def cast_to_categorical(matched_lf: pl.LazyFrame, col: str) -> pl.LazyFrame:
    """
    단일 컬럼을 Categorical로 변환
    """
    if col not in matched_lf.columns:
        print(f"[WARN] Column '{col}' not found. Skipped.")
        return matched_lf


    return matched_lf.with_columns(
        pl.col(col).cast(pl.Categorical)
    )

scan_preprocess = cast_to_categorical(scan_preprocess, "event_type")

/var/folders/qk/10b0q2j971j6d2py2x39ys200000gn/T/ipykernel_40667/1566419851.py:6: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if col not in matched_lf.columns:


## boolean
- True / False로

### bool처리 + I까지 
- boolean

adverse_event_flag

product_problem_flag

report_to_fda

manufacturer_link_flag

In [60]:
def cast_flags_to_bool(matched_lf: pl.LazyFrame, flag_cols: list[str]) -> pl.LazyFrame:
    """
    Y/N flag를 Boolean으로 변환
    - 'Y' → True
    - 'N' → False
    - 'I', null, 기타 → None  # ← "I" 처리 추가
    """
    return matched_lf.with_columns([
        pl.when(pl.col(col) == "Y")
          .then(True)
        .when(pl.col(col) == "N")
          .then(False)
        .otherwise(None)  # ← 이미 "I"를 None으로 처리된 것을 확인
        .alias(col)
        for col in flag_cols
        if col in matched_lf.columns
    ])

# 사용 (한 번만!) 
matched_lf = cast_flags_to_bool(matched_lf, [
    "adverse_event_flag",
    "manufacturer_link_flag",
    "product_problem_flag",
    "report_to_fda"  # ← "I"도 자동으로 None 처리됨
])

/var/folders/qk/10b0q2j971j6d2py2x39ys200000gn/T/ipykernel_40667/2602053894.py:16: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if col in matched_lf.columns


In [61]:
# def cast_flags_to_bool(
#     matched_lf: pl.LazyFrame,
#     flag_cols: list[str] = ["adverse_event_flag", "manufacturer_link_flag", "product_problem_flag", "report_to_fda"],
# ) -> pl.LazyFrame:
#     return matched_lf.with_columns([
#         pl.when(pl.col(col) == "Y")
#           .then(pl.lit(True))
#           .when(pl.col(col) == "N")
#           .then(pl.lit(False))
#           .otherwise(None)
#           .alias(col)
#         for col in flag_cols
#         if col in matched_lf.columns
#     ])

In [62]:
# after_vc = (
#     matched_lf.with_columns(
#         pl.when(pl.col("report_to_fda") == "I")
#           .then(None)
#           .otherwise(pl.col("report_to_fda"))
#           .alias("report_to_fda")
#     )
#     .select(pl.col("report_to_fda").value_counts(sort=True))
#     .unnest("report_to_fda")
#     .sort("count", descending=True)
#     .collect()
# )

# after_vc


### device_0_device_operator
- boolean

In [63]:
def device_operator_clean(matched_lf, col_name):
    """
    device_0_device_operator 컬럼 클린징 함수
    "HEALTH PROFESSIONAL"을 1, "LAY USER/PATIENT"을 0, 그 외는 null로 변환
    """
    return matched_lf.with_columns(
        pl.when(pl.col(col_name) == "HEALTH PROFESSIONAL")
        .then(True)
        .when(pl.col(col_name) == "LAY USER/PATIENT")
        .then(False)
        .otherwise(None)
        .cast(pl.Boolean)
        .alias(col_name)
    )

### reprocessed_and_reused_flag
- boolean

In [64]:
def convert_reprocessed_flag(matched_lf, verbose=True):
    """
    reprocessed_and_reused_flag를 Boolean 타입으로 변환
    
    처리 방법:
    - 원본 컬럼을 직접 덮어쓰기 (boolean)
    - 'Y' -> True (재처리됨)
    - 'N' -> False (재처리 안 됨)
    - null -> null
    
    Parameters:
    -----------
    matched_lf : polars.LazyFrame
        입력 데이터
    verbose : bool
        값 확인 결과 출력 여부
    
    Returns:
    --------
    polars.LazyFrame
        변환된 데이터 (원본 컬럼 덮어쓰기)
    """
    # # 값 확인
    # if verbose:
    #     print("\nreprocessed_and_reused_flag 값 확인:")
    #     value_counts = matched_lf.select(
    #         pl.col('reprocessed_and_reused_flag').value_counts().sort('count', descending=True)
    #     ).collect()
    #     print(value_counts)
    #     total = matched_lf.select(pl.len()).collect().item()
    #     null_count = matched_lf.filter(pl.col('reprocessed_and_reused_flag').is_null()).select(pl.len()).collect().item()
    #     print(f"\n전체: {total:,}개, null: {null_count:,}개 ({null_count/total*100:.2f}%)")
    
    # Boolean 타입으로 변환
    return matched_lf.with_columns([
        pl.when(pl.col('reprocessed_and_reused_flag') == pl.lit('Y'))
            .then(pl.lit(True))
        .when(pl.col('reprocessed_and_reused_flag') == pl.lit('N'))
            .then(pl.lit(False))
        .otherwise(None)
        .cast(pl.Boolean)
        .alias('reprocessed_and_reused_flag')
    ])

### single_use_flag
- boolean

In [65]:
def convert_single_use_flag(matched_lf, verbose=True):
    """
    single_use_flag를 Boolean 타입으로 변환
    
    처리 방법:
    - 원본 컬럼을 직접 덮어쓰기 (boolean)
    - 'Y' -> True (일회용)
    - 'N' -> False (다회용)
    - 그 외 -> null (알 수 없음)
    
    Parameters:
    -----------
    matched_lf : polars.LazyFrame
        입력 데이터
    verbose : bool
        값 확인 결과 출력 여부
    
    Returns:
    --------
    polars.LazyFrame
        변환된 데이터 (원본 컬럼 덮어쓰기)
    """
    # 값 확인
    # if verbose:
    #     print("\nsingle_use_flag 값 확인:")
    #     value_counts = matched_lf.select(
    #         pl.col('single_use_flag').value_counts().sort('count', descending=True)
    #     ).collect()
    #     print(value_counts)
    #     total = matched_lf.select(pl.len()).collect().item()
    #     null_count = matched_lf.filter(pl.col('single_use_flag').is_null()).select(pl.len()).collect().item()
    #     print(f"\n전체: {total:,}개, null: {null_count:,}개 ({null_count/total*100:.2f}%)")
    
    # Boolean 타입으로 변환
    return matched_lf.with_columns([
        pl.when(pl.col('single_use_flag')== pl.lit('Y'))
            .then(pl.lit(True))
        .when(pl.col('single_use_flag')== pl.lit('N'))
            .then(pl.lit(False))
        .otherwise(None)
        .cast(pl.Boolean)
        .alias('single_use_flag')
    ])

### previous_use_code
- boolean

In [66]:
def convert_previous_use_code(matched_lf, verbose=True):
    """
    previous_use_code를 boolean 타입으로 변환
    
    변환 규칙:
    - 'I' -> 'Initial' (처음 사용) -> True
    - 'R' -> 'Reuse' (재사용) -> False
    - 그 외 -> 'Unknown' (알 수 없음)
    
    참고:
    - 원본 컬럼을 직접 덮어쓰기 (boolean 타입)
    
    Parameters:
    -----------
    matched_lf : polars.LazyFrame
        입력 데이터
    verbose : bool
        값 확인 결과 출력 여부
    
    Returns:
    --------
    polars.LazyFrame
        변환된 데이터 (원본 컬럼 덮어쓰기)
    """
    # # 값 확인
    # if verbose:
    #     print("\nprevious_use_code 값 확인:")
    #     value_counts = matched_lf.select(
    #         pl.col('previous_use_code').value_counts().sort('count', descending=True)
    #     ).collect()
    #     print(value_counts)
    #     total = matched_lf.select(pl.len()).collect().item()
    #     null_count = matched_lf.filter(pl.col('previous_use_code').is_null()).select(pl.len()).collect().item()
    #     print(f"\n전체: {total:,}개, null: {null_count:,}개 ({null_count/total*100:.2f}%)")
    
    # boolean 타입으로 변환
    return matched_lf.with_columns(
        pl.when(pl.col('previous_use_code') == pl.lit('I'))
            .then(pl.lit('True'))
        .when(pl.col('previous_use_code') == pl.lit('R'))
            .then(pl.lit('False'))
        .otherwise(pl.lit('Unknown'))
        .cast(pl.Boolean)
        .alias('previous_use_code')
    )

### event_location
- bool

In [67]:
def cast_flags_to_bool(
    matched_lf: pl.LazyFrame,
    flag_cols: list[str] = ['event_location'],
) -> pl.LazyFrame:
    return matched_lf.with_columns([
        pl.when(pl.col(col) == "Y")
          .then(pl.lit(True))
          .when(pl.col(col) == "N")
          .then(pl.lit(False))
          .otherwise(None)
          .alias(col)
        for col in flag_cols
        if col in matched_lf.columns
    ])

## int, string, date

### 날짜 컬럼 (date)

date_of_event

date_recived 
  
device_date_of_manufacturer

In [68]:
import polars as pl

def cast_date_cols_safe(
    matched_lf: pl.LazyFrame,
    date_cols: list[str],
    fmt: str = "%Y%m%d",
) -> pl.LazyFrame:
    schema = matched_lf.collect_schema()
    exprs = []

    for col in date_cols:
        if col not in schema:
            continue

        dt = schema[col]

        # 이미 Date면 건드리지 않음
        if dt == pl.Date:
            continue

        # Datetime이면 Date로만 다운캐스트
        if dt == pl.Datetime:
            exprs.append(pl.col(col).cast(pl.Date, strict=False).alias(col))
            continue

        # 그 외(Utf8/Int 등)만 fmt로 파싱
        exprs.append(
            pl.col(col)
              .cast(pl.Utf8)
              .str.strptime(pl.Date, format=fmt, strict=False)
              .alias(col)
        )

    return matched_lf if not exprs else matched_lf.with_columns(exprs)


In [69]:
# # 확인 코드
# date_cols = [
#     "date_of_event",
#     "date_received",
#     "device_date_of_manufacturer",
# ]

# scan_preprocess = cast_date_cols_safe(scan_preprocess, date_cols=date_cols)


# {c: scan_preprocess.schema[c] for c in [
#     "date_of_event",
#     "date_received",
#     "device_date_of_manufacturer"
# ]}

# scan_preprocess.select([
#     pl.len().alias("n_rows"),
#     pl.col("date_of_event").is_null().sum().alias("n_null_event"),
#     pl.col("date_of_event").drop_nulls().head(5).alias("sample"),
# ]).collect()


### mdr_report_key (int 32?)

In [70]:
def cast_to_int32(
    matched_lf : pl.lazyframe,
    int_col : list[str] = ["mdr_text_key"],
) -> pl.LazyFrame : 
    return matched_lf.withcolumns([
        pl.col(col).cast(pl.Int32, strict=False).alias(col)
        for col in int_col
        if col in matched_lf.columns
    ])

# categorical 인코딩

In [71]:
def sequence_number_outcome_clean(matched_lf, col_name):
    """
    patient_0_sequence_number_outcome 컬럼을 one hot encoding (LazyFrame 유지)
    """
    
    outcome_mapping = {
        'Life Threatening': 'L',
        'Hospitalization': 'H',
        'Disability': 'S',
        'Congenital Anomaly': 'C',
        'Required Intervention': 'R',
        'Death': 'D',
        'Other': 'O',
        'Invalid Data': 'O',
        'Unknown': 'O',
        'No Information': 'O',
        'Not Applicable': 'O',
    }
    
    # 모든 가능한 outcome 코드
    all_outcomes = ['L', 'H', 'S', 'C', 'R', 'D', 'O']
    
    result = matched_lf.with_columns(
        pl.col(col_name)
        .str.replace_all(r'^\[|\]$', "")
        .str.replace_all(r"'", "")
        .str.split(",")
        .list.eval(pl.element().str.strip_chars())
        .alias("_outcome_list")
    )
    
    # 매핑 적용
    for key, value in outcome_mapping.items():
        result = result.with_columns(
            pl.col("_outcome_list")
            .list.eval(pl.element().str.replace(key, value))
            .alias("_outcome_list")
        )
    
    # 각 outcome에 대해 one-hot 컬럼 생성
    for outcome in all_outcomes:
        result = result.with_columns(
            pl.col("_outcome_list")
            .list.contains(outcome)
            .cast(pl.Int32)
            .alias(f"outcome_{outcome}")
        )
    
    result = result.drop("_outcome_list", col_name)
    
    return result

In [72]:
# 상위 10개 행만 처리해서 확인
test_result = (
    sequence_number_outcome_clean(matched_lf.head(100), "patient_0_sequence_number_outcome")
    .collect()
)

test_result.select(['outcome_L', 'outcome_H', 'outcome_S', 'outcome_C', 'outcome_R', 'outcome_D', 'outcome_O'])

outcome_L,outcome_H,outcome_S,outcome_C,outcome_R,outcome_D,outcome_O
i32,i32,i32,i32,i32,i32,i32
null,null,null,null,null,null,null
null,null,null,null,null,null,null
null,null,null,null,null,null,null
null,null,null,null,null,null,null
null,null,null,null,null,null,null
…,…,…,…,…,…,…
null,null,null,null,null,null,null
null,null,null,null,null,null,null
null,null,null,null,null,null,null


# Flag (품질 & 관련성 파악) + 파생변수

### product_problem_flag x event_type

In [73]:
matched_lf = matched_lf.with_columns(
    pl.when(pl.col("event_type").is_in(["Death", "Injury"]))
      .then(True)
      .otherwise(False)
      .alias("adverse_event_flag_logic")
)


In [74]:
# 새 컬럼 이름 "adverse_event_flag_logic"

cols = ["event_type", "adverse_event_flag", "adverse_event_flag_logic"]

# 1) True 샘플 (Death/Injury)
true_df = (
    matched_lf.filter(pl.col("adverse_event_flag_logic") == True)
      .select(cols)
      .collect()
)
true_samples = true_df.sample(n=min(20, true_df.height), seed=42)

# 2) False 샘플 (Death/Injury 제외 전부)
false_df = (
    matched_lf.filter(pl.col("adverse_event_flag_logic") == False)
      .select(cols)
      .collect()
)
false_samples = false_df.sample(n=min(20, false_df.height), seed=42)

# 3) 충돌 샘플 (원본 flag가 있고, 논리 파생값과 다른 경우)
conflict_df = (
    matched_lf.filter(
        pl.col("adverse_event_flag").is_not_null()
        & (pl.col("adverse_event_flag") != pl.col("adverse_event_flag_logic"))
    )
    .select(cols)
    .collect()
)
conflict_samples = conflict_df.sample(n=min(20, conflict_df.height), seed=42)

true_samples, false_samples.to_pandas(), conflict_samples.to_pandas()


(shape: (20, 3)
 ┌────────────┬────────────────────┬──────────────────────────┐
 │ event_type ┆ adverse_event_flag ┆ adverse_event_flag_logic │
 │ ---        ┆ ---                ┆ ---                      │
 │ str        ┆ bool               ┆ bool                     │
 ╞════════════╪════════════════════╪══════════════════════════╡
 │ Injury     ┆ true               ┆ true                     │
 │ Injury     ┆ true               ┆ true                     │
 │ Injury     ┆ true               ┆ true                     │
 │ Injury     ┆ true               ┆ true                     │
 │ Injury     ┆ true               ┆ true                     │
 │ …          ┆ …                  ┆ …                        │
 │ Injury     ┆ true               ┆ true                     │
 │ Injury     ┆ true               ┆ true                     │
 │ Injury     ┆ true               ┆ true                     │
 │ Injury     ┆ true               ┆ true                     │
 │ Injury     ┆ true    

In [75]:
# matched_lf.schema

# 텍스트 처리

# 2차 열 drop

# 추상 베이스 클래스 (UUDI)